In [ ]:
# This code was written by Luka Skolc (ETHZ) under the supervision of Krzysztof Barczynski (PMOD/WRC & ETHZ),
# Nils Janitzek (PMOD/WRC & ETHZ) and Louise Harra (PMOD/WRC & ETHZ) in the scope of the ETH Studio 
# Davos Internship in 2023. The work on the code began in March 2023.

In [1]:
##############         DO NOT CHANGE THESE FUNCTIONS WITHOUT A VERY GOOD REASON         ################

# find_brightenings_rel(intensities, t, means, threshold, smooth, delay)
# check_neighbour(a, b)
# update(shape, intensities, means, t, ref, threshold, smooth, delay)
# add_objects(objects, t, intensities, means, threshold, smooth, delay)
# expand_objects(objects, t, intensities, means, threshold, smooth, delay)
# join_objects(objects, t)
# filter_onthego(objects, t, min_time, min_size, max_size)

def find_brightenings_rel(intensities, t, means, threshold, smooth, delay):
    masked = np.where(intensities[:, :, t] < threshold * means[:, :, t - smooth - delay], 0, intensities[:, :, t])
    # Ignore all points that do not significantly brighten up and change their values to 0
    indices = np.nonzero(masked)  # Get the coordinates of all the points that significantly brighten up
    #print('indices = {}'.format(indices))
    
    number = indices[0].size  #  How many points brighten up
    adjacent_positions = set() # Here we will store all pairs of adjacent points. They are represented
                               # by their position in the "indices" array (scalars)

    for i in range(number - 1):
        if indices[0][i] == indices[0][i+1]:  # y coordinates are equal
            if np.abs(indices[1][i] - indices[1][i+1]) == 1:
                #print('first condition')
                adjacent_positions.add((i, i+1))

        elif indices[1][i] == indices[1][i+1]:  # x coordinates are equal
            if np.abs(indices[0][i] - indices[0][i+1]) == 1:
                #print('second condition')
                adjacent_positions.add((i, i+1))

    sort = np.argsort(indices[1][:])  # We want the x values to be ascending
    # Taking the indices array along the array sort will ensure points are ordered by their x coordinates
    indices_alt = np.array([np.take_along_axis(indices[0][:], sort, axis=0), np.take_along_axis(indices[1][:], sort, axis=0)])

    for i in range(number - 1):
        if indices_alt[0][i] == indices_alt[0][i+1]:  # y coordinates are equal
            if np.abs(indices_alt[1][i] - indices_alt[1][i+1]) == 1:
                #print('alt first condition')
                adjacent_positions.add((sort[i], sort[i+1]))
                # we want the indices in the original array, not the resorted one

        elif indices_alt[1][i] == indices_alt[1][i+1]:  # x coordinates are equal
            if np.abs(indices_alt[0][i] - indices_alt[0][i+1]) == 1:
                #print('alt second condition')
                adjacent_positions.add((sort[i], sort[i+1]))

    brightening_points = set() # Coordinates of points whose neighbours also brighten up

    for pair in adjacent_positions: # We only add the first point from each pair
        brightening_points.add((indices[0][pair[0]], indices[1][pair[0]]))
    
    return brightening_points
    
    # Note we might have not included all points in the pairs from 'adjacent positions'. This is not
    # an issue as we will be building connected graphs anyway


def check_neighbour(a, b):  # Checks if a and b are adjacent pixels
    if a[0] == b[0] and np.abs(a[1] - b[1]) == 1: # The pixels are horizontally adjacent
        return True
    
    if a[1] == b[1] and np.abs(a[0] - b[0]) == 1: # The pixels are vertically adjacent
        return True
    
    return False


def update(shape, intensities, means, t, ref, threshold, smooth, delay):  # Expand a shape at time t-1
    # into "new_shape" at time t.
    # "ref" is the time at which the parent object was created. The function 'update' does not know 
    # about other objects.

    new_shape = set()
    
    (ny, nx, nt) = intensities.shape
    
    for coord in shape: # Checks which of the previous bright points make it to the next frame.
        # Intensities at time t are compared to the mean value "delay" frames before the object appeared.
        y0 = coord[0]
        x0 = coord[1]
        if intensities[y0, x0, t] > threshold * means[y0, x0, ref - smooth - delay]:
            new_shape.add((y0, x0))
    
    while(1): # Keep expanding new_shape until there is something to add
        added = 0
        for coord in shape.union(new_shape): # "new_shape" can outgrow "shape"
            y0 = coord[0]
            x0 = coord[1]
            
            if x0 + 1 < nx: # so that we don't try to go outside the array and raise index errors
                if (y0, x0 + 1) not in new_shape:
                    if intensities[y0, x0 + 1, t] > threshold * means[y0, x0 + 1, ref - smooth - delay]:
                        new_shape.add((y0, x0 + 1))
                        added += 1
                        
            if x0 - 1 > -1:
                if (y0, x0 - 1) not in new_shape:
                    if intensities[y0, x0 - 1, t] > threshold * means[y0, x0 - 1, ref - smooth - delay]:
                        new_shape.add((y0, x0 - 1))
                        added += 1
            
            if y0 + 1 < ny:
                if (y0 + 1, x0) not in new_shape:
                    if intensities[y0 + 1, x0, t] > threshold * means[y0 + 1, x0, ref - smooth - delay]:
                        new_shape.add((y0 + 1, x0))
                        added += 1
            
            if y0 - 1 > -1:
                if (y0 - 1, x0) not in new_shape:
                    if intensities[y0 - 1, x0, t] > threshold * means[y0 - 1, x0, ref - smooth - delay]:
                        new_shape.add((y0 - 1, x0))
                        added += 1
            
        if added == 0: 
            break
    
    return new_shape   # If an empty set is returned, the object is no longer active and we will 
                       # terminate its evolution in the 'objects' list.


def add_objects(objects, t, intensities, means, threshold, smooth, delay): 
    # Find brightenings at time t and create new objects if the brightenings are disjoint from the 
    # existing objects.
    
    brightenings = find_brightenings_rel(intensities, t, means, threshold, smooth, delay) 
    # The "rel" method gets less (irrelevant) objects and is more flexible than the "diff" method
    
    for point in brightenings: # Go over all the points that brighten up
        counter = 0  # Counter set to 1 if point is included in any object
        for obj in objects:
            if (obj[0] + len(obj[1:]) - 1)  <  t: # If the object had faded away, we don't consider it
                continue                                # here we also consider newly added objects
                
            for obj_point in obj[-1]:
                if check_neighbour(obj_point, point): # Intuitively, this condition should never occur
                    counter = 1  # because we already expanded the objects. But the criteria are different!
                    break  # For brightening we take the mean from time t, while for expanding the same 
                    # point we take the mean at the ref time of the object - the point can brighten
                    # w.r.t. one mean but not the other
                
                if obj_point == point:  # The brightening is inside an existing object
                    counter = 1
                    break
                    
            if counter == 1: break

        if counter == 0: # Create maximal object from new point
            new_object = update({point}, intensities, means, t, t, threshold, smooth, delay)
            counter2 = 0  
            # Removing the lower part should not change the number of objects, only possibly enlarge some
            '''for obj in objects: # a matter of debate what to do here: we can create the object regardless  
                # and let it join the overlapping parent object later - this will mean we miss less objects
                if (obj[0] + len(obj[1:]) - 1)  <  t: # consider active (here t2 = t) and newly added objects
                    continue
                if new_object.intersection(obj[-1]) != set():
                    counter2 = 1
                    break'''
            if counter2 == 0 and new_object != set():
                objects.append([t, new_object])


def expand_objects(objects, t, intensities, means, threshold, smooth, delay): 
    # Expand all active objects 
    for obj in objects:
        if (obj[0] + len(obj[1:]) - 1) <  (t - 1): # If the object isn't active (t2 = t-1), we don't
            continue                               # consider it. 
        
        obj_update = update(obj[-1], intensities, means, t, obj[0], threshold, smooth, delay)
        # The 'ref' time is set to obj[0], the time at which the object was created.
        if obj_update == set(): # object is no longer active
            continue
            
        obj.append(obj_update)

        
def join_objects(objects, t): # Join objects that overlap into a single object with a combined history.
    no = len(objects)
    deleted = [] # A list of indices of objects to be removed. Contains no duplicates.
    
    for i in range(no):  # The construction is always such that j < i, therefore object[j] started at 
        if i in deleted: # an earlier or equal time than object[i] because objects are appended to the 
            continue     # end of the list when they are created.
        
        if (objects[i][0] + len(objects[i][1:]) - 1) != t: 
            continue
            
        for j in range(i):
            if j in deleted: 
                continue
                
            if (objects[j][0] + len(objects[j][1:]) - 1) != t: 
                continue
                
            if objects[i][-1].intersection(objects[j][-1]) != set(): # The two objects overlap
                if i in deleted: # We need this condition because otherwise the same index i may be
                    continue     # added to the "deleted" list twice
                    
                deleted.append(i)
                ti1 = objects[i][0] # By construction, tj1 <= ti1 so we always add to the j-th object 
                tj1 = objects[j][0] # and delete the i-th object
                
                for k in range(1, 1 + len(objects[i][1:])):
                    objects[j][-k].update(objects[i][-k])

    for p in sorted(deleted, reverse = True): # Remove the objects. The "reverse = True" argument
        del objects[p]                        # ensures that the correct objects are deleted.


def filter_onthego(objects, t, min_time, min_size, max_size):
    no = len(objects)
    removed2 = [] # A list of indices of objects to be removed.
    
    for i in range(no):
        if (objects[i][0] + len(objects[i][1:]) - 1) != t: # The object is not active anymore
            if len(objects[i]) <= min_time: # Too short-lived
                removed2.append(i)
                
            elif len(time_union(objects[i])) < min_size: # Too small time union
                removed2.append(i)
                
            elif len(time_union(objects[i])) > max_size: # Too large time union
                removed2.append(i)

    for p in sorted(removed2, reverse = True):
        del objects[p]

